In [1]:
import os
from os.path import join
from dotenv import load_dotenv
import random, string

import pyspark as ps
from pyspark.sql import SparkSession, types
from pyspark.sql import functions as F
from pyspark.sql.functions import col

In [2]:
ps_conf = ps.SparkConf()\
            .set("spark.sql.sources.commitProtocolClass", "org.apache.spark.sql.execution.datasources.SQLHadoopMapReduceCommitProtocol")\
            .set("mapreduce.fileoutputcommitter.marksuccessfuljobs", "false")\
            .set("spark.sql.shuffle.partitions", 100)\
            .set("spark.sql.dynamicPartitionPruning.enabled", True)
            # '_started'と'_committed_'で始まるファイルを書き込まないように設定
            # '_SUCCESS'で始まるファイルを書き込まないように設定
            # パーティション数を調整する
            # 動的パーティションプルーニングの有効化
spark = SparkSession.builder.config(conf=ps_conf).getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/09 18:40:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
load_dotenv(join(os.getcwd(), '.env'))
BASE_PATH = os.environ.get("BASE_PATH")
WORK_PATH = BASE_PATH + os.environ.get("WORK_PATH")

In [ ]:
# 回遊1階層のテストデータ生成

# 75端末、24時間分
unit_id_num = 75*24

random_unit_list = [''.join(random.choices(string.digits, k=5)) for _ in range(0, unit_id_num)]
tmp_list = [[unit_id, random.random()] for unit_id in random_unit_list]

df_schema = types.StructType([
        types.StructField('ORIGIN',      types.StringType(), False),
        types.StructField('移動影響量',    types.FloatType(),  False),
    ])
df_migrate1 = spark.createDataFrame(tmp_list, df_schema)
df_migrate1\
	.orderBy(col('ORIGIN').asc())\
    .toPandas()\
    .to_csv(WORK_PATH + 'csv_data/test_1.csv', index=False, header=True)

In [6]:
# 回遊2階層のテストデータ生成

df_schema = types.StructType([
        types.StructField('ORIGIN',      types.StringType(), False),
        types.StructField('DESTINATION', types.StringType(), False),
        types.StructField('移動影響量',    types.FloatType(),  False),
    ])
df_migrate2 = spark.createDataFrame([], df_schema)

for unit_id, move in tmp_list:
    df_tmp      = df_migrate1\
        				.withColumn('DESTINATION', F.lit(unit_id))\
						.withColumn('移動影響量',    col('移動影響量') * move)
    df_migrate2 = df_migrate2.unionByName(df_tmp)

df_migrate2\
    .orderBy(col('ORIGIN').asc(), col('DESTINATION').asc())\
    .toPandas()\
    .to_csv(WORK_PATH + 'csv_data/test_2.csv', index=False, header=True)

24/12/09 18:40:56 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
Exception in thread "serve-DataFrame" java.net.SocketTimeoutException: Accept timed out
	at java.base/sun.nio.ch.NioSocketImpl.timedAccept(NioSocketImpl.java:701)
	at java.base/sun.nio.ch.NioSocketImpl.accept(NioSocketImpl.java:745)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:695)
	at java.base/java.net.ServerSocket.platformImplAccept(ServerSocket.java:660)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:636)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:582)
	at java.base/java.net.ServerSocket.accept(ServerSocket.java:541)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:65)
